In [1]:
# Let's explore some agents

In [2]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm.invoke("How many letters in the word educa?")

AIMessage(content='There are 6 letters in the word "educa".')

In [3]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Return the length of a word."""
    return len(word)

tools = [get_word_length]

In [4]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're a powerful assistant, but bad at calculating lengths of words."),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [17]:
from langchain.tools.render import format_tool_to_openai_function

# Binding passes in the tools each time we invoke the model
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

# Thoughts: Does this use up the context?

In [22]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [29]:
response = agent.invoke({"input": "How many letters in the word educa?", "intermediate_steps": []})
response

AgentActionMessageLog(tool='get_word_length', tool_input={'word': 'educa'}, log="\nInvoking: `get_word_length` with `{'word': 'educa'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "word": "educa"\n}', 'name': 'get_word_length'}})])

In [41]:
# But we need a runtime in order to be able to handle the agents and executing functions.

from langchain.schema.agent import AgentFinish

# The world's most basic runtime
user_input = "how many letters in the word educa? and how many letters in the word praya?"
intermediate_steps = []
while True:
    output = agent.invoke(
        {
            "input": user_input,
            "intermediate_steps": intermediate_steps,
        }
    )

    # If we encounter the finish
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:  # If we don't, we continue
        print(f"OUTPUT: {output}")
        print(f"TOOL NAME: {output.tool}")
        print(f"TOOL INPUT: {output.tool_input}")
        tool = {"get_word_length": get_word_length}[output.tool]  # this is ugly, but it'll do for now because we only have one tool
        observation = tool.run(output.tool_input)
        intermediate_steps.append((output, observation))
print(final_result)

OUTPUT: tool='get_word_length' tool_input={'word': 'educa'} log="\nInvoking: `get_word_length` with `{'word': 'educa'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "word": "educa"\n}', 'name': 'get_word_length'}})]
TOOL NAME: get_word_length
TOOL INPUT: {'word': 'educa'}
OUTPUT: tool='get_word_length' tool_input={'word': 'praya'} log="\nInvoking: `get_word_length` with `{'word': 'praya'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "word": "praya"\n}', 'name': 'get_word_length'}})]
TOOL NAME: get_word_length
TOOL INPUT: {'word': 'praya'}
The word "educa" has 5 letters, and the word "praya" also has 5 letters.


In [42]:
# Luckily there's ome included executors
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [44]:
agent_executor.invoke(
    {"input": "how many letters in the world educa? What about the word superaya?"}
)



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5
Invoking: `get_word_length` with `{'word': 'superaya'}`


8The word "educa" has 5 letters, while the word "superaya" has 8 letters.

> Finished chain.


{'input': 'how many letters in the world educa? What about the word superaya?',
 'output': 'The word "educa" has 5 letters, while the word "superaya" has 8 letters.'}

In [45]:
# All right, let's add memory to make things stateful.

from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You are a powerful assistant, but bad at calculating lengths of words."
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

In [46]:
# Let's keep track of the chat history
from langchain.schema.messages import AIMessage, HumanMessage

chat_history = []

In [47]:
# Let's put it together
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

[]

In [49]:
# Let's do some magic
input1 = "how many letters in the word educa?"
result = agent_executor.invoke(
    {
        "input": input1, "chat_history": chat_history
    }
)

chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)

# Execute a second step
agent_executor.invoke(
    {
        "input": "is that a real word?",
        "chat_history": chat_history
    }
)



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5The word "educa" is not a real word. It was just used as an example to calculate the number of letters in a word.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".')],
 'output': 'The word "educa" is not a real word. It was just used as an example to calculate the number of letters in a word.'}

## Let's see if we can wire this up to an openbb function
(Even if it's hardcoded for now)


- Agent
- Tools
- Prompt
- AgentExecutor (runtime)


In [5]:
import inspect

from openbb import obb
from openbb_core.app.router import CommandMap
from openbb_core.app.provider_interface import ProviderInterface

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import tool, Tool, AgentExecutor, AgentOutputParser
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.tools import StructuredTool

In [7]:
from pydantic.v1 import create_model, BaseModel
from pydantic.v1.fields import FieldInfo
from pydantic_core import PydanticUndefinedType

Goal: let's make it easy to convert an OpenBB function --> Langchain / openai function.

Schema comes from the ProviderInterface  
Function / Endpoint map comes from CommandMap + scraping the Members

In [9]:
def _fetch_obb_module(openbb_command_root):
    module_path_split = openbb_command_root.split("/")[1:]
    module_path = '.'.join(module_path_split)

    # Iteratively get module
    module = obb
    for attr in module_path.split('.'):
        module=getattr(module, attr)

    return module

def _fetch_schemas(openbb_command_root):
    # Ugly hack to make it compatiable
    # (even though we convert it back)
    # so that we have a nicer API.
    module_root_path = openbb_command_root.replace("/", ".")
    schemas = {
        k.replace('.', '/'): v for k, v in obb.coverage.command_model.items() if module_root_path in k
    }
    return schemas

def _fetch_callables(openbb_command_root):
    module = _fetch_obb_module(openbb_command_root)
    members = inspect.getmembers(module)
    members_dict = {x[0]: x[1] for x in members if '__' not in x[0] and '_run' not in x[0]}

    schemas = _fetch_schemas(openbb_command_root)

    # Create callables dict, with the same key as used in the schemas
    callables = {}
    for k in schemas.keys():
        try:
            callables[k] = members_dict[k.split('/')[-1]] 
        except KeyError:  # Sometimes we don't have a specific callable for an endpoint, so we skip.
            pass
    return callables

fundamental_schemas = _fetch_schemas('/equity/fundamental')
fundamental_callables = _fetch_callables('/equity/fundamental')

In [97]:
from typing import get_args
def _fetch_outputs(schema):
    outputs = []
    output_fields = schema['openbb']['Data']['fields']
    for name, t in output_fields.items():
        if isinstance(t.annotation, type):
            type_str = t.annotation.__name__
        else:
            type_str = str(t.annotation).replace("typing.", "")
        outputs.append((name, type_str))
    return outputs

In [35]:
from typing import get_origin, Literal

def from_schema_to_pydantic_model(model_name, schema):
    create_model_kwargs = {}
    for field, field_info in schema.items():

        field_type = field_info.annotation

        # Handle default values
        if not isinstance(field_info.default, PydanticUndefinedType):
            field_default_value = field_info.default
            new_field_info = FieldInfo(  # Weird hack, because of how the default field value works
                description=field_info.description,
                default=field_default_value,
            )
        else:
            new_field_info = FieldInfo(
                description=field_info.description,
            )
        create_model_kwargs[field] = (field_type, new_field_info)
    return create_model(model_name, **create_model_kwargs)

In [38]:
# Time for a little decorator magic (even though we won't use it as a decorator

def return_results(func):
    def wrapper_func(*args, **kwargs):
        return func(*args, **kwargs).results
    return wrapper_func


In [129]:
def from_openbb_to_langchain_func(openbb_callable, openbb_schema):
    func_name = openbb_callable.__name__
    func_schema = openbb_schema['openbb']['QueryParams']['fields']
    
    pydantic_model = from_schema_to_pydantic_model(
        model_name=f'{func_name}InputModel',
        schema=func_schema
    )

    outputs = _fetch_outputs(openbb_schema)
    description = openbb_callable.__doc__.split("\n")[0]
    description += "\nThe following data is available:\n\n"
    description += "\n".join(e[0] for e in outputs)

    tool = StructuredTool(
        name = func_name,
        func=return_results(openbb_callable),
        description=description,
        args_schema=pydantic_model
    )

    return tool

In [130]:
def map_openbb_functions_to_langchain_tools(schemas_dict, callables_dict):
    tools = []
    for route in callables_dict.keys():
        tool = from_openbb_to_langchain_func(
            openbb_callable=callables_dict[route],
            openbb_schema=schemas_dict[route]
        )
        tools.append(tool)
    return tools

In [131]:
def map_openbb_collection_to_langchain_tools(openbb_command_root):
    schemas = _fetch_schemas(openbb_command_root)
    callables = _fetch_callables(openbb_command_root)
    tools = map_openbb_functions_to_langchain_tools(
        schemas_dict=schemas,
        callables_dict=callables
    )
    return tools

In [157]:
fundamental_tools = map_openbb_collection_to_langchain_tools("/equity/fundamental")
compare_tools = map_openbb_collection_to_langchain_tools("/equity/compare")

In [208]:
# Let's add some memory
MEMORY_KEY = "chat_history"

# Create our prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You're a powerful financial advisor.
            Do your best to answer the questions,
            and make use of the tools provided, paying special attention to the outputs they provide."""
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        (
            "user",
            "{input}",
        ),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

# Bind our tools
# llm = ChatOpenAI(model="gpt-4-1106-preview")
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in compare_tools])


# Create the agent
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"]
    }
    | prompt
    | llm
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=compare_tools, verbose=True)

In [206]:
chat_history = []

In [207]:
from langchain.schema.messages import AIMessage, HumanMessage

human_input = "Why is Sony a competitor for AAPL?"

response = agent_executor.invoke(
    {
        "input": human_input, "chat_history": chat_history
    }
    
)

chat_history.extend(
    [
        HumanMessage(content=human_input),
        AIMessage(content=response["output"])
    ]
)

print(response['output'])



> Entering new AgentExecutor chain...
Sony Corporation and Apple Inc. (AAPL) are competitors in various segments of the electronics and entertainment industries. Here are the primary reasons why Sony is considered a competitor to Apple:

1. **Consumer Electronics**: Both companies produce a range of consumer electronics. Apple is well-known for its iPhones, iPads, Mac computers, Apple Watch, and other devices. Sony, on the other hand, offers its own laptops, tablets, and smartphones, although it has a smaller market share in these areas compared to Apple.

2. **Audio Products**: Sony has a strong presence in the audio industry, with products like headphones, earbuds, and home audio systems. Apple, with its AirPods, HomePod, and the acquisition of Beats, also competes in the high-end audio segment.

3. **Gaming**: Sony's PlayStation consoles compete with Apple's gaming offerings, which are part of its App Store ecosystem. While Apple does not produce dedicated gaming consoles, its dev

# Let's explore different agents

In [222]:
from langchain import hub
from langchain.agents import initialize_agent
prompt = hub.pull("hwchase17/react")

In [200]:
response['chat_history']

[HumanMessage(content="Who are AAPL's competitors?"),
 AIMessage(content='Apple Inc. (AAPL) competes with a variety of companies in different segments of the electronics and technology markets. Some of its competitors include:\n\n- LG Display Co Ltd (LPL)\n- Sony Group Corporation (SNEJF)\n- Panasonic Corporation (PCRFY)\n- Sonos, Inc. (SONO)\n- Vizio Holding Corp (VZIO)\n- mCloud Technologies Corp (MICS)\n- Worlds Inc (WLDSW)\n- Koss Corporation (KOSS)\n- GoPro, Inc. (GPRO)\n- Sony Corporation (SONY)\n- Universal Electronics Inc. (UEIC)\n- Turtle Beach Corporation (HEAR)\n- Vuzix Corporation (VUZI)\n- Worlds Inc. (WLDS)\n\nPlease note that the competitive landscape for Apple is broad and includes many other companies not listed here, due to its diverse product range including smartphones, computers, tablets, wearables, and services.'),
 HumanMessage(content='Why is Sony a competitor for AAPL?'),
 AIMessage(content="Sony Corporation is considered a competitor to Apple Inc. (AAPL) for s